In [1]:
%load_ext autoreload
from __future__ import print_function, division

In [2]:
%autoreload

import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import torch, torch.utils.data as utils, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter


from mmd_grud_utils2 import *

In [3]:
DATA_FILEPATH     = '../data/all_hourly_data.h5'
# RAW_DATA_FILEPATH = '/scratch/mmd/mimic_data/final/nogrouping_5/all_hourly_data.h5'
GAP_TIME          = 6  # In hours
WINDOW_SIZE       = 24 # In hours
SEED              = 1
ID_COLS           = ['subject_id', 'hadm_id', 'icustay_id']
GPU               = '2'

os.environ['CUDA_VISIBLE_DEVICES'] = GPU
np.random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
class DictDist():
    def __init__(self, dict_of_rvs): self.dict_of_rvs = dict_of_rvs
    def rvs(self, n):
        a = {k: v.rvs(n) for k, v in self.dict_of_rvs.items()}
        out = []
        for i in range(n): out.append({k: vs[i] for k, vs in a.items()})
        return out
    
class Choice():
    def __init__(self, options): self.options = options
    def rvs(self, n): return [self.options[i] for i in ss.randint(0, len(self.options)).rvs(n)]

In [5]:
%%time
data_full_lvl2 = pd.read_hdf(DATA_FILEPATH, 'vitals_labs')
# data_full_raw  = pd.read_hdf(RAW_DATA_FILEPATH, 'vitals_labs') 
statics        = pd.read_hdf(DATA_FILEPATH, 'patients')

CPU times: total: 19.8 s
Wall time: 38.9 s


In [6]:
data_full_lvl2.head()

LEVEL2                                 alanine aminotransferase             \
Aggregation Function                                      count  mean  std   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                             2.0  25.0  0.0   
                              1                             0.0   NaN  NaN   
                              2                             0.0   NaN  NaN   
                              3                             0.0   NaN  NaN   
                              4                             0.0   NaN  NaN   

LEVEL2                                 albumin           albumin ascites       \
Aggregation Function                     count mean  std           count mean   
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0            2.0  1.8  0.0             0.0  NaN   
                              1            0.0  NaN  NaN             0.0  NaN   
                              2            0.0  NaN  NaN             0.0  NaN   
                              3            0.0  NaN  NaN             0.0  NaN   
                              4            0.0  NaN  NaN             0.0  NaN   

LEVEL2                                     albumin pleural  ...  \
Aggregation Function                   std           count  ...   
subject_id hadm_id icustay_id hours_in                      ...   
3          145834  211552     0        NaN             0.0  ...   
                              1        NaN             0.0  ...   
                              2        NaN             0.0  ...   
                              3        NaN             0.0  ...   
                              4        NaN             0.0  ...   

LEVEL2                                 white blood cell count  \
Aggregation Function                                      std   
subject_id hadm_id icustay_id hours_in                          
3          145834  211552     0                      4.012837   
                              1                           NaN   
                              2                           NaN   
                              3                           NaN   
                              4                           NaN   

LEVEL2                                 white blood cell count urine           \
Aggregation Function                                          count mean std   
subject_id hadm_id icustay_id hours_in                                         
3          145834  211552     0                                 0.0  NaN NaN   
                              1                                 0.0  NaN NaN   
                              2                                 0.0  NaN NaN   
                              3                                 0.0  NaN NaN   
                              4                                 0.0  NaN NaN   

LEVEL2                                    ph                 ph urine           
Aggregation Function                   count  mean       std    count mean std  
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0          9.0  7.40  0.147733      1.0  5.0 NaN  
                              1          0.0   NaN       NaN      0.0  NaN NaN  
                              2          3.0  7.26  0.000000      0.0  NaN NaN  
                              3          0.0   NaN       NaN      0.0  NaN NaN  
                              4          0.0   NaN       NaN      0.0  NaN NaN  

[5 rows x 312 columns]

In [7]:
# data_full_raw.head()

In [8]:
statics.head()

,,,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,discharge_location,fullcode_first,dnr_first,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,SNF,1.0,0.0,...,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0,0,0,1,0,145
4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,HOME WITH HOME IV PROVIDR,1.0,0.0,...,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0,0,0,1,0,40
6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,HOME HEALTH CARE,1.0,0.0,...,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0,0,0,1,0,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,DEAD/EXPIRED,1.0,0.0,...,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1,1,1,1,0,127
11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,HOME HEALTH CARE,1.0,0.0,...,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0,0,0,1,0,38


In [9]:
def simple_imputer(df):
    idx = pd.IndexSlice
    df = df.copy()
    if len(df.columns.names) > 2: df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))
    
    df_out = df.loc[:, idx[:, ['mean', 'count']]]
    icustay_means = df_out.loc[:, idx[:, 'mean']].groupby(ID_COLS).mean()
    
    df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(
        method='ffill'
    ).groupby(ID_COLS).fillna(icustay_means).fillna(0)
    
    df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
    df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
    
    is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
    hours_of_absence = is_absent.cumsum()
    time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
    time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)

    df_out = pd.concat((df_out, time_since_measured), axis=1)
    df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)
    
    df_out.sort_index(axis=1, inplace=True)
    return df_out

In [10]:
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['mort_hosp', 'mort_icu', 'los_icu']]
Ys['los_3'] = Ys['los_icu'] > 3
Ys['los_7'] = Ys['los_icu'] > 7
Ys.drop(columns=['los_icu'], inplace=True)
Ys.astype(float)

lvl2 = data_full_lvl2[
    (data_full_lvl2.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))) &
    (data_full_lvl2.index.get_level_values('hours_in') < WINDOW_SIZE)
] 

# raw.columns = raw.columns.droplevel(level=['label', 'LEVEL1', 'LEVEL2'])

train_frac, dev_frac, test_frac = 0.7, 0.1, 0.2
lvl2_subj_idx, Ys_subj_idx = [df.index.get_level_values('subject_id') for df in (lvl2, Ys)]
lvl2_subjects = set(lvl2_subj_idx)
assert lvl2_subjects == set(Ys_subj_idx), "Subject ID pools differ!"
# assert lvl2_subjects == set(raw_subj_idx), "Subject ID pools differ!"

np.random.seed(SEED)
subjects, N = np.random.permutation(list(lvl2_subjects)), len(lvl2_subjects)
N_train, N_dev, N_test = int(train_frac * N), int(dev_frac * N), int(test_frac * N)
train_subj = subjects[:N_train]
dev_subj   = subjects[N_train:N_train + N_dev]
test_subj  = subjects[N_train+N_dev:]

[(lvl2_train, lvl2_dev, lvl2_test), (Ys_train, Ys_dev, Ys_test)] = [
    [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj, dev_subj, test_subj)] \
    for df in (lvl2, Ys)
]

idx = pd.IndexSlice
lvl2_means, lvl2_stds = lvl2_train.loc[:, idx[:,'mean']].mean(axis=0), lvl2_train.loc[:, idx[:,'mean']].std(axis=0)
# raw_means, raw_stds = raw_train.loc[:, idx[:,'mean']].mean(axis=0), raw_train.loc[:, idx[:,'mean']].std(axis=0)

lvl2_train.loc[:, idx[:,'mean']] = (lvl2_train.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds
lvl2_dev.loc[:, idx[:,'mean']] = (lvl2_dev.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds
lvl2_test.loc[:, idx[:,'mean']] = (lvl2_test.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds

# raw_train.loc[:, idx[:,'mean']] = (raw_train.loc[:, idx[:,'mean']] - raw_means)/raw_stds
# raw_dev.loc[:, idx[:,'mean']] = (raw_dev.loc[:, idx[:,'mean']] - raw_means)/raw_stds
# raw_test.loc[:, idx[:,'mean']] = (raw_test.loc[:, idx[:,'mean']] - raw_means)/raw_stds

In [11]:
lvl2_train, lvl2_dev, lvl2_test = [
    simple_imputer(df) for df in (lvl2_train, lvl2_dev, lvl2_test)
]
lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test = [
    df.pivot_table(index=['subject_id', 'hadm_id', 'icustay_id'], columns=['hours_in']) for df in (
        lvl2_train, lvl2_dev, lvl2_test
    )
]

for df in lvl2_train, lvl2_dev, lvl2_test: assert not df.isnull().any().any()

C:\Users\kisse\AppData\Local\Temp\ipykernel_13600\2730890864.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
C:\Users\kisse\AppData\Local\Temp\ipykernel_13600\2730890864.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
C:\Users\kisse\AppData\Local\Temp\ipykernel_13600\2730890864.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pand

In [12]:
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['mort_hosp', 'mort_icu', 'los_icu']]
Ys['los_3'] = Ys['los_icu'] > 3
Ys['los_7'] = Ys['los_icu'] > 7
Ys.drop(columns=['los_icu'], inplace=True)
Ys.astype(float)
[(Ys_train, Ys_dev, Ys_test)] = [
    [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj, dev_subj, test_subj)] \
    for df in (Ys,)
]

### Task Prediction

#### Hyperparams

In [13]:
N = 10

GRU_D_dist = DictDist({
    'cell_size': ss.randint(50, 75),
    'hidden_size': ss.randint(65, 95), 
    'learning_rate': ss.uniform(2e-3, 1e-1),
    'num_epochs': ss.randint(15, 150),
    'patience': ss.randint(3, 7),
    'batch_size': ss.randint(35, 65),
    'early_stop_frac': ss.uniform(0.05, 0.1),
    'seed': ss.randint(1, 10000),
})
np.random.seed(SEED)
GRU_D_hyperparams_list = GRU_D_dist.rvs(N)

# with open('/scratch/mmd/extraction_baselines_gru-d.pkl', mode='rb') as f: results = pickle.load(f)

In [14]:
results = {}

In [21]:
for i, h in enumerate(GRU_D_hyperparams_list):
    GRU_D_hyperparams_list[i]['batch_size'] = int(GRU_D_hyperparams_list[i]['batch_size'])

In [25]:
GRU_D_hyperparams_list

[{'cell_size': 55,
  'hidden_size': 66,
  'learning_rate': 0.07052195003967596,
  'num_epochs': 22,
  'patience': 6,
  'batch_size': 60,
  'early_stop_frac': 0.051828827734419186,
  'seed': 9496},
 {'cell_size': 61,
  'hidden_size': 77,
  'learning_rate': 0.022445224973151746,
  'num_epochs': 78,
  'patience': 5,
  'batch_size': 38,
  'early_stop_frac': 0.12501443149449676,
  'seed': 3429},
 {'cell_size': 62,
  'hidden_size': 72,
  'learning_rate': 0.08981174363909455,
  'num_epochs': 76,
  'patience': 4,
  'batch_size': 39,
  'early_stop_frac': 0.14888610889064946,
  'seed': 156},
 {'cell_size': 58,
  'hidden_size': 78,
  'learning_rate': 0.004738759319792616,
  'num_epochs': 37,
  'patience': 3,
  'batch_size': 59,
  'early_stop_frac': 0.12481656543798395,
  'seed': 4410},
 {'cell_size': 59,
  'hidden_size': 93,
  'learning_rate': 0.06904675101784023,
  'num_epochs': 72,
  'patience': 5,
  'batch_size': 52,
  'early_stop_frac': 0.07804439920644052,
  'seed': 649},
 {'cell_size': 61,


### GRU-D

In [24]:
model_name       = 'GRU-D'
hyperparams_list = GRU_D_hyperparams_list
RERUN            = False
if model_name not in results: results[model_name] = {}
for t in ['mort_icu', 'los_3', 'mort_hosp', 'los_7']:
    if t not in results[model_name]: results[model_name][t] = {}
    for n, X_train, X_dev, X_test in (
        ('lvl2', lvl2_train, lvl2_dev, lvl2_test),
#         ('raw', raw_train, raw_dev, raw_test)
    ):
        print("Running model %s on target %s with representation %s" % (model_name, t, n))
        X_mean = np.nanmean(
            to_3D_tensor(
                X_train.loc[:, pd.IndexSlice[:, 'mean']] * 
                np.where((X_train.loc[:, pd.IndexSlice[:, 'mask']] == 1).values, 1, np.NaN)
            ),
            axis=0, keepdims=True
        ).transpose([0, 2, 1])
        base_params = {'X_mean': X_mean, 'output_last': True, 'input_size': X_mean.shape[2]}
    
        if n in results[model_name][t]:
            if not RERUN: 
                print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                print(results[model_name][t][n])
                continue
            best_s, best_hyperparams = results[model_name][t][n][-1], results[model_name][t][n][1]
            print("Loading best hyperparams", best_hyperparams)
        else:
            best_s, best_hyperparams = -np.Inf, None
            for i, hyperparams in enumerate(hyperparams_list):
                print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))

                early_stop_frac,batch_size,seed = [hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]

                np.random.seed(seed)
                all_train_subjects = list(
                    np.random.permutation(Ys_train.index.get_level_values('subject_id').values)
                )
                N_early_stop        = int(len(all_train_subjects) * early_stop_frac)
                train_subjects      = all_train_subjects[:-N_early_stop]
                early_stop_subjects = all_train_subjects[-N_early_stop:]
                X_train_obs         = X_train[X_train.index.get_level_values('subject_id').isin(train_subjects)]
                Ys_train_obs        = Ys_train[Ys_train.index.get_level_values('subject_id').isin(train_subjects)]

                X_train_early_stop  = X_train[X_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
                Ys_train_early_stop = Ys_train[
                    Ys_train.index.get_level_values('subject_id').isin(early_stop_subjects)
                ]

                train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=int(batch_size))
                early_stop_dataloader = prepare_dataloader(
                    X_train_early_stop, Ys_train_early_stop[t], batch_size=int(batch_size)
                )
                dev_dataloader        = prepare_dataloader(X_dev, Ys_dev[t], batch_size=int(batch_size))
                test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=int(batch_size))

                model_hyperparams = copy.copy(base_params)
                model_hyperparams.update(
                    {k: v for k, v in hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
                )
                model = GRUD(**model_hyperparams)

                best_model, _ = Train_Model(
                    model, train_dataloader, early_stop_dataloader,
                    **{k: v for k, v in hyperparams.items() if k in (
                        'num_epochs', 'patience', 'learning_rate', 'batch_size'
                    )}
                )

                probabilities_dev, labels_dev = predict_proba(best_model, dev_dataloader)
                probabilities_dev = np.concatenate(probabilities_dev)[:, 1]
                labels_dev        = np.concatenate(labels_dev)
                s = roc_auc_score(labels_dev, probabilities_dev)
                if s > best_s:
                    best_s, best_hyperparams = s, hyperparams
                    print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))
                
        ## Test
        np.random.seed(seed)
        hyperparams = best_hyperparams # In case I forgot a replace below
        early_stop_frac,batch_size,seed = [best_hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
        
        X_train_concat, Ys_train_concat = pd.concat((X_train, X_dev)), pd.concat((Ys_train, Ys_dev))
        
        all_train_subjects = list(np.random.permutation(Ys_train_concat.index.get_level_values('subject_id').values))
        N_early_stop = int(len(all_train_subjects) * early_stop_frac)
        train_subjects, early_stop_subjects = all_train_subjects[:-N_early_stop], all_train_subjects[-N_early_stop:]
        X_train_obs         = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(train_subjects)]
        Ys_train_obs        = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(train_subjects)]

        X_train_early_stop  = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]
        Ys_train_early_stop = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]

        train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
        early_stop_dataloader = prepare_dataloader(X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size)
        test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

        model_hyperparams = copy.copy(base_params)
        model_hyperparams.update(
            {k: v for k, v in best_hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
        )
        model = GRUD(**model_hyperparams)

        best_model, (losses_train, losses_early_stop, losses_epochs_train, losses_epochs_early_stop) = Train_Model(
            model, train_dataloader, early_stop_dataloader,
            **{k: v for k, v in best_hyperparams.items() if k in (
                'num_epochs', 'patience', 'learning_rate', 'batch_size'
            )}
        )

        probabilities_test, labels_test = predict_proba(best_model, test_dataloader)

        y_score = np.concatenate(probabilities_test)[:, 1]
        y_pred  = np.argmax(probabilities_test)
        y_true  = np.concatenate(labels_test)

        auc   = roc_auc_score(y_true, y_score)
        auprc = average_precision_score(y_true, y_score)
        acc   = accuracy_score(y_true, y_pred)
        F1    = f1_score(y_true, y_pred)
        print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
        print(auc, auprc, acc, F1)
        
        results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
        with open('../src/model/extraction_baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)

Running model GRU-D on target mort_icu with representation lvl2


C:\Users\kisse\OneDrive\Documents\GitHub\FuzzyMedNet\notebooks\mmd_grud_utils2.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))
C:\Users\kisse\AppData\Local\Temp\ipykernel_13600\1457725132.py:12: RuntimeWarning: Mean of empty slice
  X_mean = np.nanmean(


On sample 1 / 10 (hyperparams = {'cell_size': 55, 'hidden_size': 66, 'learning_rate': 0.07052195003967596, 'num_epochs': 22, 'patience': 6, 'batch_size': 60, 'early_stop_frac': 0.051828827734419186, 'seed': 9496})
Model Structure:  GRUD(
  (zl): Linear(in_features=274, out_features=66, bias=True)
  (rl): Linear(in_features=274, out_features=66, bias=True)
  (hl): Linear(in_features=274, out_features=66, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=66, bias=True)
  (fc): Linear(in_features=66, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: nan, valid_loss: nan, time: [33.52], best model: 1
Epoch: 1, train_loss: nan, valid_loss: nan, time: [32.36], best model: 0
Epoch: 2, train_loss: nan, valid_loss: nan, time: [32.

ValueError: Input contains NaN.

In [ ]:
y_score = np.concatenate(probabilities_test)[:, 1]
y_pred  = np.concatenate(probabilities_test).argmax(axis=1)
y_true  = np.concatenate(labels_test)

auc   = roc_auc_score(y_true, y_score)
auprc = average_precision_score(y_true, y_score)
acc   = accuracy_score(y_true, y_pred)
F1    = f1_score(y_true, y_pred)
print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
print(auc, auprc, acc, F1)

results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
with open('../src/model/extraction_baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)